# Benchmark de los modelos QUBO para el problema de la mochila
A continuación, analizamos las distintas soluciones al problema QUBO que hemos diseñado, para compararlas.

## Soluciones a estudiar
- QUBO con variables no binarias y holgura
- QUBO con variables no binarias sin holgura
- QUBO con variables no binarias, sin holgura, creando paquetes de acciones
- QUBO con variables no binarias, sin holgura, creando paquetes de acciones, con un máximo por activo
- QUBO iterativo con variables no binarias, sin holgura, creando paquetes cada vez más pequeños

## Parámetros a medir
- Número de variables
- Tiempo de ejecución
- Retorno total
- Riesgo total
- Total invertido

## Problemas a estudiar
- Problema estático modelo (el usado durante el desarrollo)
- Cartera sobre un conjunto de 10 acciones reales

## Reutilizando el trabajo previo
En primer lugar, importamos todas las modelizaciones QUBO que hemos creado en el apartado anterior

In [ ]:
import io
import os
import sys
import ast
import json
import types

def import_notebook_code(fullname):
    """
    Importa un notebook (.ipynb) como un módulo, pero elimina automáticamente todas las celdas Markdown y
    evita ejecutar código que no sea definición de clases/funciones.
    """
    path = fullname if fullname.endswith(".ipynb") else f"{fullname}.ipynb"
    
    if not os.path.exists(path):
        raise FileNotFoundError(f"No se encuentra el notebook: {path}")

    with open(path, 'r', encoding='utf-8') as f:
        nb = json.load(f)

    mod = types.ModuleType(fullname)
    sys.modules[fullname] = mod
    
    for cell in nb['cells']:
        if cell['cell_type'] == 'code':
            source = "".join(cell['source'])
            try:
                tree = ast.parse(source)
                # Si la celda no contiene al menos una definición (class/def/import), la saltamos
                if not any(isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.Import, ast.ImportFrom)) for node in tree.body):
                     # Permitimos asignaciones simples (constantes)
                     if not all(isinstance(node, (ast.Assign, ast.AnnAssign)) for node in tree.body):
                         continue 
            except:
                pass # Si hay error de sintaxis, ignoramos y dejamos que exec falle o pase
            
            try:
                exec(source, mod.__dict__)
            except Exception as e:
                print(f"Aviso: Error en una celda del notebook importado: {e}")

    return mod

optimizacion_carteras = import_notebook_code('02-optimizacion_carteras')


# Clases Principales
FinancialAsset = optimizacion_carteras.FinancialAsset
DecisionVariable = optimizacion_carteras.DecisionVariable
PortfolioProblem = optimizacion_carteras.PortfolioProblem
PortfolioSolution = optimizacion_carteras.PortfolioSolution

add_method = optimizacion_carteras.add_method

# Funciones de Resolución (Solvers)
solve_qubo_basico_vars = optimizacion_carteras.solve_qubo_basico_vars
solve_qubo_basico_risk = optimizacion_carteras.solve_qubo_basico_risk

# Función para generar problemas aleatorios
create_random_problem = optimizacion_carteras.create_random_problem

# Problemas de ejemplo
problema_random = optimizacion_carteras.problema_random
problema_estatico = optimizacion_carteras.problema_estatico


## Preparación de Datos para Optimización de Cartera QUBO

El objetivo de esta fase es transformar los precios históricos brutos en los tres parámetros clave requeridos por nuestro modelo QUBO (Optimización Cuadrática Binaria Sin Restricciones).

Para ello se utiliza un enfoque de Backtesting, donde la optimización se realiza con datos de un período pasado (In-Sample).

Los tres inputs esenciales que obtendremos de la historia de precios son:



1.   Retorno Esperado ($r_i$): El beneficio esperado de cada activo.
2.   Coste ($c_i$): El precio de compra en el momento de la optimización.
3.   Matriz de Covarianza ($\sigma_{ij}$): El componente de riesgo que mide cómo se mueven los activos entre sí.

El siguiente código Python descarga y calcula estos parámetros utilizando un período de entrenamiento fijo (2023-2025) para simular una decisión de inversión tomada a principios de 2025.

In [ ]:

import yfinance as yf
import numpy as np

# --- 1. Definición de Parámetros de Entrada para Backtesting ---

# Tickers de activos (Sectores y Geografías)
# ^GSPC: S&P 500 (Índice USA) | RY: Royal Bank of Canada (Finanzas Canadá)
# NEE: NextEra Energy (Utilities/Energía USA) | DIS: Walt Disney Co (Entretenimiento)
# BABA: Alibaba (Tech China) | SNEJF: Sony Group Corp (Tech/Electro Japón)

TICKERS = ['^GSPC', 'RY', 'XOM', 'DIS', 'BABA', 'TLT','NVDA', 'NVO', 'NU', 'GLD','SNEJF', 'NEE']
TICKERS = [ 'RY', 'XOM', 'DIS', 'TLT','NVDA']


START_DATE = '2023-01-01'
END_DATE = '2025-01-01'
INTERVALO_ANALISIS = '1d' # Diario


# --- 2. Descarga de Datos Históricos (Datasheet Crudo) ---

datos_crudos = yf.download(TICKERS,
                           start=START_DATE,
                           end=END_DATE,
                           interval=INTERVALO_ANALISIS,
                           auto_adjust=True)

precios = datos_crudos['Close'].dropna()


# --- 3. Derivación de Parámetros Clave para el Modelo (a 2025-01-01) ---

## PARÁMETRO 1: Coste (c_i)

# El coste c_i es el precio más reciente antes de la fecha de inversión (END_DATE).
costes_ci = precios.iloc[-1].rename('Coste ($c_i$)')


## PARÁMETRO 2: Retornos Diarios y Retorno Esperado (r_i)

# Calcular los retornos diarios logarítmicos
retornos = np.log(precios / precios.shift(1)).dropna()

# Multiplicamos por 252 (días de trading en un año) para anualizar.
roi = retornos.mean() * 252
roi.rename('Retorno Esperado ($r_i$ Anualizado)', inplace=True)


## PARÁMETRO 3: Matriz de Covarianza (Sigma_ij)

# La Matriz de Covarianza (Sigma) se calcula a partir de los retornos, anualizada
sigma_matrix = retornos.cov() * 252

print(f"\n- **Fecha de Optimización (Entrada del Modelo):** {END_DATE}")
print(f"\n- **r_i (Retorno Esperado):**\n{roi}")
print(f"\n- **c_i (Coste/Precio):**\n{costes_ci}")
print(f"\n- **Matriz Sigma (Covarianza):**\n{sigma_matrix}")

## Construcción del problema con los activos seleccionados
A continuación, construimos un problema con los datos reales de acciones. 

In [ ]:
# Presupuesto máximo: 10.000
problema_real = PortfolioProblem(10000)

for name in TICKERS:
    price = costes_ci.to_dict()[name]
    variance = sigma_matrix.loc[name, name]**2
    roi_ratio = roi.to_dict()[name]
    problema_real.add_asset(name, price, variance, roi_ratio)

for n1 in TICKERS:
    for n2 in TICKERS:
        if n1 > n2:
            problema_real.add_covariance(n1,n2,sigma_matrix.loc[n1, n2])

problema_real.print_problem_definition(display_variables = False, display_covariances = True)


## Ejecutamos el benchmark para todos los métodos y problemas



In [ ]:
import time
from pprint import pprint

soluciones_problema_estatico = {}

# - QUBO con variables no binarias
# - QUBO con variables no binarias sin holgura
# - QUBO con variables no binarias, sin holgura, creando paquetes de acciones
# - QUBO con variables no binarias, sin holgura, creando paquetes de acciones, con un máximo por activo
# - QUBO iterativo con variables no binarias, sin holgura, creando paquetes cada vez más pequeños




def run_benchmark(problema, name, skip=0, max_weight_per_asset = 0.5, min_package_pct=0.12):
    print ("\n\n","----------------------------------")
    print ("Ejecutando: ",name)
    print ("----------------------------------")
    problema.clear_variables()
    problema.print_problem_definition()
    name_ = name + "_"
    problemas = {}
    soluciones_problema = {}
    problema_1 = problema.copy()
    problema_1.create_variables_binary_simple()
    problema_1.create_slack_variables()
    
    problemas[name_+'multivariable y holgura'] = problema_1
    
    problema_2 = problema.copy()
    problema_2.create_variables_binary_simple()
    problemas[name_+'multivariable sin holgura'] = problema_2
    
    problema_3 = problema.copy()
    problema_3.create_variables_binary_packaged(min_package_pct)
    k = name_+'multivariable paquetizado ' + str(round(min_package_pct*100, 2)) + "%"
    problemas[k] = problema_3
    print("\n\n" + k)
    problema_3.print_problem_definition(True,True)
    
    problema_3b = problema.copy()
    min_package_pct_reduced = min_package_pct/2
    problema_3b.create_variables_binary_packaged(min_package_pct_reduced)
    k = name_+'multivariable paquetizado ' + str(round(min_package_pct_reduced*100, 2)) + "%"
    problemas[k] = problema_3b
    print("\n\n" + k)
    problema_3b.print_problem_definition(True,True)

    problema_4 = problema.copy()
    problema_4.create_variables_with_max_weight(max_weight_per_asset, min_package_pct, fill_in_mode = -1)
    k = name_+'multivariable paquetizado ' + str(round(min_package_pct*100, 2)) + "%" + " max: " + str(round(max_weight_per_asset*100,2)) + "%"
    problemas[k] = problema_4
    print("\n\n" + k)
    problema_4.print_problem_definition(True,True)
    
    
    for key, p in problemas.items():
        if skip > 0:
            skip = skip -1
            solution = PortfolioSolution(num_variables = len(p.variables))
            soluciones_problema[key] = solution
            continue
        print("\n\n","-----------")
        print("Ejecutando ",key)
        print("-----------")
        inicio = time.process_time()
        solution = solve_qubo_basico_risk(p, print_solution = False)
        fin = time.process_time()
        solution.run_time = fin - inicio
        soluciones_problema[key] = solution
        p.print_solution(solution.solucion)

    print("\n\n","-----------", "ITERATIVA")
    #Ahora la solución iterativa
    min_package_pct_orig = min_package_pct
    min_package_pct_1 = min_package_pct*3/4
    min_package_pct_2 = min_package_pct_1/8

    problema_5 = problema.copy()
    problema_5.create_variables_binary_packaged(min_package_pct_1)
    problema_5.print_problem_definition(True,True)

    key = name_+'iterativo multiv. paq. ' + str(round(min_package_pct_1*100, 2)) + "% - " + str(round(min_package_pct_2*100, 2)) + "%"
    problemas[key] = problema_5

    print("\n\n","-----------")
    print("Ejecutando ",key)
    print("-----------")
    inicio = time.process_time()
    solution1 = solve_qubo_basico_risk(problema_5, print_solution=False)
    fin = time.process_time()
    solution1.run_time = fin - inicio
    problema_5.print_solution(solution1.solucion)
    
    solution_table = problema_5.get_solution_table(solution1.solucion)
    remaining_budget = problema_5.max_budget
    bounds = {}
    for name, a in solution_table.items():
        remaining_budget -= a['cost']
        min_amount = a['cost'] - min_package_pct_1*problema_5.max_budget
        max_amount = a['cost'] + min_package_pct_1*problema_5.max_budget
        bounds[name] = (min_amount, max_amount)
    max_holgura = remaining_budget + min_package_pct_1*problema_5.max_budget
    
    problema_5b = problema_5.copy()
    problema_5b.clear_variables()
    problema_5b.create_variables_with_bounds(bounds,min_package_pct_2)
    
    problema_5b.print_problem_definition(True,True)
    print("\n\n","-----------")
    print("Ejecutando Run 2 ",key)
    print("-----------")
    inicio = time.process_time()
    solution2 = solve_qubo_basico_risk(problema_5b, solution1.alpha, solution1.beta, print_solution=False)
    fin = time.process_time()
    solution2.run_time = solution1.run_time + (fin - inicio)
    solution2.num_variables = max(solution1.num_variables, solution2.num_variables)
    soluciones_problema[key] = solution2
    problema_5b.print_solution(solution2.solucion)



    print("\n\n","-----------", "ITERATIVA HOLG")
    #Ahora la solución iterativa
    min_package_pct_orig = min_package_pct
    min_package_pct_1 = min_package_pct*4/5
    min_package_pct_2 = min_package_pct_1/8

    problema_6 = problema.copy()
    problema_6.create_variables_binary_packaged(min_package_pct_1)
    problema_6.create_slack_variables_packaged(min_package_pct_1, budget_limit=problema_6.max_budget*0.5)
    problema_6.print_problem_definition(True,True)

    key = name_+'iterativo multiv. paq. holg ' + str(round(min_package_pct_1*100, 2)) + "% - " + str(round(min_package_pct_2*100, 2)) + "%"
    problemas[key] = problema_6

    print("\n\n","-----------")
    print("Ejecutando ",key)
    print("-----------")
    inicio = time.process_time()
    solution1 = solve_qubo_basico_risk(problema_6, print_solution=False)
    fin = time.process_time()
    solution1.run_time = fin - inicio
    problema_6.print_solution(solution1.solucion)
    
    solution_table = problema_6.get_solution_table(solution1.solucion)
    remaining_budget = problema_6.max_budget
    bounds = {}
    for name, a in solution_table.items():
        remaining_budget -= a['cost']
        min_amount = a['cost'] - min_package_pct_1*problema_6.max_budget
        max_amount = a['cost'] + min_package_pct_1*problema_6.max_budget
        bounds[name] = (min_amount, max_amount)
    max_holgura = remaining_budget + min_package_pct_1*problema_6.max_budget
    
    problema_6b = problema_6.copy()
    problema_6b.clear_variables()
    problema_6b.create_variables_with_bounds(bounds,min_package_pct_2)
    problema_6b.create_slack_variables_packaged(min_package_pct_2, budget_limit=max_holgura)
    
    problema_6b.print_problem_definition(True,True)
    print("\n\n","-----------")
    print("Ejecutando Run 2 ",key)
    print("-----------")
    inicio = time.process_time()
    solution2 = solve_qubo_basico_risk(problema_6b, solution1.alpha, solution1.beta, print_solution=False)
    fin = time.process_time()
    solution2.run_time = solution1.run_time + (fin - inicio)
    solution2.num_variables = max(solution1.num_variables, solution2.num_variables)
    soluciones_problema[key] = solution2
    problema_6b.print_solution(solution2.solucion)
    
    return (problemas, soluciones_problema)



(problemas_1, soluciones_1) = run_benchmark(problema_estatico, name="Ej", max_weight_per_asset = 0.5, min_package_pct=0.10)

for k,s in soluciones_1.items():
    print(k)
    pprint(vars(s))

(problemas_2, soluciones_2) = run_benchmark(problema_real, name="Real", skip=2, max_weight_per_asset = 0.5, min_package_pct=0.08)

for k,s in soluciones_2.items():
    print(k)
    pprint(vars(s))

In [ ]:
import pandas as pd

def tabla_comparativa(soluciones):
    data = []

    for nombre, s in soluciones.items():
        # Extraemos los atributos del objeto de solución 's'
        run_time = getattr(s, 'run_time', 0.0)
        if run_time is not None:
            run_time = round(run_time,4)
        retorno = getattr(s, 'retorno', 0.0)
        if retorno is not None:
            retorno = round(retorno,2)
        riesgo = getattr(s, 'riesgo', 0.0)
        if riesgo is not None:
            riesgo = round(riesgo,0)
        inversion = getattr(s, 'inversion', 0.0)
        if inversion is not None:
            inversion = round(inversion,2)
             
        fila = {
            "Configuración": nombre,
            "N Variables": getattr(s, 'num_variables', 'N/A'),
            "Tiempo (s)": run_time,
            "Retorno Total": retorno,
            "Riesgo Total": riesgo,
            "Total Invertido": inversion
        }

        data.append(fila)

    df = pd.DataFrame(data)
    return df


df_ejemplo = tabla_comparativa(soluciones_1)
df_real = tabla_comparativa(soluciones_2)

display(df_ejemplo)
display(df_real)